In [1]:
from mysql.connector import MySQLConnection, Error

In [2]:
def make_connection():
    """
    Make a connection to the school database.
    """
    try:
        conn = MySQLConnection(host = 'IES-ADS-ClassDB.sjsu.edu',
                               database = 'querycrew_wh',
                               user = 'querycrew_user',
                               password = 'Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [5]:
conn = make_connection()
cursor = conn.cursor()

Connected to the MySQL database!


In [7]:
sql = "use querycrew_wh;"

cursor.execute(sql)
#rows = cursor.fetchall()

In [9]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout,
    QPushButton, QLineEdit, QLabel, QTabWidget, QMessageBox, QTableView
)
from PyQt5.QtSql import QSqlDatabase, QSqlTableModel

class OperationalApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Operational Database GUI")
        self.resize(800, 600)

        # Initialize database connection
        self.db = self.init_db()

        # Set up a model for the 'Customers' table
        self.model = QSqlTableModel(self, self.db)
        self.model.setTable("Customers")
        self.model.select()

        # Build the UI
        self.init_ui()

    def init_db(self):
        db = QSqlDatabase.addDatabase("QMYSQL")
        db.setHostName("IES-ADS-ClassDB.sjsu.edu")
        db.setDatabaseName("querycrew_wh")
        db.setUserName("querycrew_user")
        db.setPassword("Pomegranate_746")
        if not db.open():
            QMessageBox.critical(None, "DB Connection Error", db.lastError().text())
            sys.exit(1)
        return db

    def init_ui(self):
        tabs = QTabWidget()

        # --- Customers Tab ---
        cust_tab = QWidget()
        cust_layout = QVBoxLayout()

        # Table View
        view = QTableView()
        view.setModel(self.model)
        cust_layout.addWidget(view)

        # Input Form to add a new customer
        form_layout = QHBoxLayout()
        self.first_name_input = QLineEdit()
        form_layout.addWidget(QLabel("First Name:"))
        form_layout.addWidget(self.first_name_input)
        # Add other fields similarly: last_name, email, etc.

        add_btn = QPushButton("Add Customer")
        add_btn.clicked.connect(self.add_customer)
        form_layout.addWidget(add_btn)
        cust_layout.addLayout(form_layout)

        cust_tab.setLayout(cust_layout)
        tabs.addTab(cust_tab, "Customers")

        # You can add more tabs (e.g., Car, car_sales) following the same pattern

        self.setCentralWidget(tabs)

    def add_customer(self):
        fn = self.first_name_input.text().strip()
        if not fn:
            QMessageBox.warning(self, "Input Error", "First name cannot be empty.")
            return

        # Create a new record and populate fields
        record = self.model.record()
        record.setValue("first_name", fn)
        # record.setValue("last_name", self.last_name_input.text().strip())
        # record.setValue("email", self.email_input.text().strip())

        if not self.model.insertRecord(-1, record):
            QMessageBox.warning(self, "Insert Error", self.model.lastError().text())
        else:
            self.model.select()  # Refresh view
            self.first_name_input.clear()
            # clear other inputs similarly

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = OperationalApp()
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
cursor.close()
conn.close()